<a href="https://colab.research.google.com/github/manglesh001/DL_Assignment_2/blob/main/DLL_Ass_2_PartA_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

kevinhandry_inaturalist_path = kagglehub.dataset_download('kevinhandry/inaturalist')

print('Data source import complete.')


In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset, random_split
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
from PIL import Image
import wandb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm


In [ ]:
!pip install wandb

In [ ]:
!wandb login 6001619563748a57b4114b0bb090fd4129ba6122

In [ ]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes=10, filter_counts=[32, 64, 128, 256, 512], filter_sizes=[(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)],
                 activation_func=nn.ReLU(), fc_neurons=512, use_batch_norm=True,dropout_rate=0.2):
        super(CNNModel, self).__init__()
        #RGB channels
        in_channels = 3
        # Store layers in List
        self.features = nn.ModuleList()

        # doing 5 conv-act-maxpool blocks
        for i in range(5):
            # Add  2D convolution layer
            conv = nn.Conv2d(in_channels, filter_counts[i], kernel_size=filter_sizes[i], padding='same')
            self.features.append(conv)

            # Add batch normalization
            if use_batch_norm:
                self.features.append(nn.BatchNorm2d(filter_counts[i]))

            # Add activation function
            self.features.append(activation_func)

            # Add max pooling layer
            self.features.append(nn.MaxPool2d(kernel_size=2, stride=2))

            # Update in_channels for the next layer
            in_channels = filter_counts[i]

        # Adaptive pooling to handle different input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(filter_counts[-1], fc_neurons),activation_func,
            nn.Dropout(dropout_rate) if dropout_rate > 0 else nn.Identity(),
            nn.Linear(fc_neurons, num_classes)
        )

    def forward(self, x):
        # Pass input through feature layers
        for layer in self.features:
            x = layer(x)

        # Global average pooling
        x = self.adaptive_pool(x)

        # Flatten the tensor
        x = torch.flatten(x, 1)

        # Pass through classifier
        x = self.classifier(x)
        return x

In [ ]:
#check GPU working?
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU"


(True, 'Tesla T4')

In [ ]:
# Function to get activation function
def get_activation(name):
    if name == 'relu':
        return nn.ReLU()
    elif name == 'gelu':
        return nn.GELU()
    elif name == 'silu':
        return nn.SiLU()
    elif name == 'mish':
        return nn.Mish()
    else:
        return nn.ReLU()  # Default

In [ ]:

# Data preparation functions
# split train data into val =0.2 * train
def get_dataset_and_loaders(data_dir, train_transform, val_transform, batch_size=64, val_split=0.2):
    full_train_dataset = ImageFolder(os.path.join(data_dir, 'train'), transform=train_transform)

    # Get class indices
    classes = full_train_dataset.classes
    class_to_idx = full_train_dataset.class_to_idx

    # Create indices for stratified split
    targets = np.array(full_train_dataset.targets)

    # Split indices for each class to maintain class balance
    train_indices, val_indices = [], []

    for class_idx in range(len(classes)):
        class_indices = np.where(targets == class_idx)[0]
        # data split
        n_val = int(len(class_indices) * val_split)

        # Shuffle indices
        np.random.shuffle(class_indices)

        # Split into train and validation
        val_indices.extend(class_indices[:n_val])
        train_indices.extend(class_indices[n_val:])

    # Create train and validation subsets
    train_dataset = Subset(full_train_dataset, train_indices)
    val_dataset = Subset(full_train_dataset, val_indices)

    # For validation set, we want to use a different transform
    val_dataset.dataset.transform = val_transform

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, classes


In [ ]:



# Training function
def train_model(config=None):
    # Initialize wandb
    with wandb.init(config=config):
        # Get the configuration
        config = wandb.config

        # Set device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        # Set input size to smaller dimension to run cuda OOM
        #changes image size 160*160
        input_size = 160

        # Set up data augmentation based on config
        if config.data_aug:
            train_transform = transforms.Compose([
                transforms.Resize((input_size, input_size)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            train_transform = transforms.Compose([
                transforms.Resize((input_size, input_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

        val_transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Get data loaders
        train_loader, val_loader, classes = get_dataset_and_loaders(
            '/kaggle/input/inaturalist/inaturalist_12K',
            train_transform,
            val_transform,
            batch_size=config.train_batch_size
        )

        # Get activation function
        activation = get_activation(config.activation)

        # Create the model
        model = CNN(
            num_classes=len(classes),
            filter_counts=config.filter_counts,
            filter_sizes=config.shape_of_filters_conv,
            activation_func=activation,
            fc_neurons=config.fc_layer,
            use_batch_norm=config.batch_norm_use,
            dropout_rate=config.dropout
        )

        model.to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        # Learning rate scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        # Track best validation accuracy
        best_val_acc = 0.0

        # Training loop
        for epoch in range(config.epochs):
            # Training phase
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0

            for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Train]"):
                inputs, labels = inputs.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

            train_loss = running_loss / len(train_loader.dataset)
            train_acc = correct / total

            # Validation phase
            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0

            with torch.no_grad():
                for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Val]"):
                    inputs, labels = inputs.to(device), labels.to(device)

                    # Forward pass
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # Statistics
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = outputs.max(1)
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()

            val_loss = val_loss / len(val_loader.dataset)
            val_acc = correct / total

            # Log metrics
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc
            })

            print(f"Epoch {epoch+1}/{config.epochs}, " f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


In [ ]:
# # Configure the sweep
sweep_config = {
    'method': 'random',

    'parameters': {
        'learning_rate': {
            'values': [0.001, 0.0005, 0.0001,0.01]
        },
        'train_batch_size': {
            'values': [64, 128]
        },
        'epochs': {
            'values': [10,20]
        },
        'activation': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
        'filter_counts': {
            'values': [
                [16, 32, 64, 128, 256],
                [32, 64, 128, 256, 512],
                [32, 32, 32, 32, 32],
                [64, 64, 64, 64, 64],
                [128, 128,128, 128,128],
                [128, 64, 32, 16, 8],
                [8, 16, 32, 64, 128]
            ]
        },
        'shape_of_filters_conv': {
            'values': [
                [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)],
                [(5, 5), (5, 5), (5, 5), (5, 5), (5, 5)].
                [(7, 7), (7, 7), (7, 7), (7, 7), (7, 7)].
                [(7, 7), (5, 5), (3, 3), (3, 3), (3, 3)],
                [(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)],

            ]
        },
        'fc_layer': {
            'values': [128, 256, 512]
        },
        'batch_norm_use': {
            'values': [True, False]
        },
        'dropout': {
            'values': [0, 0.2, 0.3,0.5,0.8]
        },
        'data_aug': {
            'values': [True, False]
        }
    }
}

# Initialize wandb
wandb.login()

# Create the sweep
sweep_id = wandb.sweep(sweep_config, project='inaturalist-cnn')

# Run the sweep
wandb.agent(sweep_id, train_model, count=15)


Create sweep with ID: piq7fn15
Sweep URL: https://wandb.ai/manglesh-patidar-cs24m025/inaturalist-cnn/sweeps/piq7fn15


wandb: Agent Starting Run: 0tm0ir67 with config:
wandb: 	activation: relu
wandb: 	batch_norm_use: False
wandb: 	data_aug: False
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	fc_layer: 128
wandb: 	filter_counts: [128, 64, 32, 16, 8]
wandb: 	learning_rate: 0.0001
wandb: 	shape_of_filters_conv: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	train_batch_size: 128


Using device: cuda


Epoch 1/10 [Val]: 100%|██████████| 16/16 [00:11<00:00,  1.43it/s]


Epoch 1/10, Train Loss: 2.3107, Train Acc: 0.1036, Val Loss: 2.3030, Val Acc: 0.1001


...

Using device: cuda


Epoch 1/20 [Val]: 100%|██████████| 16/16 [00:10<00:00,  1.50it/s]


Epoch 1/20, Train Loss: 2.2237, Train Acc: 0.1966, Val Loss: 2.1493, Val Acc: 0.2286


Epoch 2/20 [Train]:  79%|███████▉  | 50/63 [00:39<00:10,  1.28it/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1243, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 114, in get
    raise Empty
_queue.Empty

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipykernel_31/1492669151.py", line 80, in train_model
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} [Train]"):
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 701, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/to

epoch,▁
train_accuracy,▁
train_loss,▁
val_accuracy,▁
val_loss,▁
epoch,1
train_accuracy,0.19662
train_loss,2.22374
val_accuracy,0.22861
val_loss,2.14927


In [ ]:

# Configure the sweep
#hyperparameter
sweep_config = {
    'method': 'bayes',

    'parameters': {
        'learning_rate': {
            'values': [0.001, 0.0005, 0.0001,0.01]
        },
        'train_batch_size': {
            'values': [64, 128]
        },
        'epochs': {
            'values': [10,20]
        },
        'activation': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
        'filter_counts': {
            'values': [
                [16, 32, 64, 128, 256],
                [32, 64, 128, 256, 512],
                [32, 32, 32, 32, 32],
                [64, 64, 64, 64, 64],
                [128, 128,128, 128,128],
                [128, 64, 32, 16, 8],
                [8, 16, 32, 64, 128]
            ]
        },
        'shape_of_filters_conv': {
            'values': [
                [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)],
                [(5, 5), (5, 5), (5, 5), (5, 5), (5, 5)].
                [(7, 7), (7, 7), (7, 7), (7, 7), (7, 7)].
                [(7, 7), (5, 5), (3, 3), (3, 3), (3, 3)],
                [(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)],

            ]
        },
        'fc_layer': {
            'values': [128, 256, 512]
        },
        'batch_norm_use': {
            'values': [True, False]
        },
        'dropout': {
            'values': [0, 0.2, 0.3,0.5,0.8]
        },
        'data_aug': {
            'values': [True, False]
        }
    }
}

# Initialize wandb
wandb.login()

# Create the sweep
sweep_id = wandb.sweep(sweep_config, project='inaturalist-cnn')

# Run the sweep
wandb.agent(sweep_id, train_model, count=15)

Create sweep with ID: vad5l02n
Sweep URL: https://wandb.ai/manglesh-patidar-cs24m025/inaturalist-cnn/sweeps/vad5l02n


wandb: Agent Starting Run: wmve8i7v with config:
wandb: 	activation: mish
wandb: 	batch_norm_use: True
wandb: 	data_aug: True
wandb: 	dropout: 0.5
wandb: 	epochs: 20
wandb: 	fc_layer: 256
wandb: 	filter_counts: [32, 32, 32, 32, 32]
wandb: 	learning_rate: 0.0005
wandb: 	shape_of_filters_conv: [[7, 7], [5, 5], [3, 3], [3, 3], [3, 3]]
wandb: 	train_batch_size: 64


Using device: cuda


Epoch 1/20 [Val]: 100%|██████████| 32/32 [00:11<00:00,  2.89it/s]


Epoch 1/20, Train Loss: 2.1753, Train Acc: 0.2036, Val Loss: 2.1083, Val Acc: 0.2426


...

Epoch 19/20 [Val]: 100%|██████████| 16/16 [00:10<00:00,  1.54it/s]


Epoch 19/20, Train Loss: 1.3294, Train Acc: 0.5355, Val Loss: 1.7582, Val Acc: 0.4067


Epoch 20/20 [Val]: 100%|██████████| 16/16 [00:10<00:00,  1.56it/s]

Epoch 20/20, Train Loss: 1.2947, Train Acc: 0.5447, Val Loss: 1.6564, Val Acc: 0.4352


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_accuracy,▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
train_loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▁▁
val_accuracy,▁▂▃▄▄▄▅▅▅▅▆▅▇▆▆▇▇▇▇█
val_loss,█▆▆▄▅▅▄▄▄▄▃▅▂▃▂▂▂▂▃▁
epoch,20
train_accuracy,0.54475
train_loss,1.29465
val_accuracy,0.43522
val_loss,1.65643
